# Lösung Lab 05: SQLAlchemy Basics

### Basis Aufgabe
Hier nutzen wir Core-Features für Setup und sichere Abfragen.

In [ ]:
from sqlalchemy import create_engine, text

# 1. Engine erstellen
engine = create_engine("sqlite:///bank.db")

# 2. Verbindung & Setup
with engine.connect() as conn:
    # Tabelle erstellen (Falls nicht existiert)
    # Hinweis: In SQL ist TEXT gängig für Strings, FLOAT für Fließkomma
    create_table_sql = """
    CREATE TABLE IF NOT EXISTS accounts (
        id INTEGER PRIMARY KEY,
        owner TEXT,
        balance FLOAT
    )
    """
    conn.execute(text(create_table_sql))
    
    # Dummy Daten einfügen (Vorsicht: bei mehrfacher Ausführung entstehen Duplikate ohne Unique Constraint)
    # Wir löschen vorher alles, um das Notebook sauber zu halten
    conn.execute(text("DELETE FROM accounts"))
    
    conn.execute(text("INSERT INTO accounts (owner, balance) VALUES ('Alice', 1000.0)"))
    conn.execute(text("INSERT INTO accounts (owner, balance) VALUES ('Bob', 50.0)"))
    
    # Wichtig: Commit bei Schreiboperationen im Core Modus
    conn.commit()
    print("Setup complete. Table 'accounts' created and populated.")

In [ ]:
# 3. Sichere Abfrage (Parameterized Query)
# Simulation einer Benutzereingabe
user_search = "Alice"

# SQL Injection Prevention: Wir nutzen :owner als Platzhalter
query = text("SELECT * FROM accounts WHERE owner = :search_name")

with engine.connect() as conn:
    # Parameter als Dictionary übergeben
    result = conn.execute(query, {"search_name": user_search})
    
    print(f"Search results for '{user_search}':")
    for row in result:
        # Zugriff via Spaltennamen (row.owner, row.balance)
        print(f"ID: {row.id} | Owner: {row.owner} | Balance: {row.balance}")

### Bonus Herausforderung
Definition von Klassen und Nutzung der Session.

In [ ]:
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column, Session

# 1. Model Definition
class Base(DeclarativeBase):
    pass

class Transaction(Base):
    __tablename__ = "transactions"
    
    # Type Hints definieren das Schema
    id: Mapped[int] = mapped_column(primary_key=True)
    amount: Mapped[float]
    purpose: Mapped[str]

# Tabelle physisch in der DB erstellen (basierend auf der Klasse)
Base.metadata.create_all(engine)

# 2. Daten Speichern mit Session
with Session(engine) as session:
    # Objekt erstellen statt SQL schreiben
    new_tx = Transaction(amount=2500.50, purpose="Salary October")
    
    # Zur Session hinzufügen
    session.add(new_tx)
    
    # Transaktion abschließen
    session.commit()
    print("Transaction object saved to database.")

# Check (Optional): Lesen um zu beweisen, dass es geklappt hat
with Session(engine) as session:
    # Einfaches raw SQL zum Prüfen oder ORM query (letzteres war nicht Teil der Angabe, aber gut zur Kontrolle)
    stmt = text("SELECT * FROM transactions")
    for row in session.execute(stmt):
        print(f"Stored Transaction: {row.purpose}, {row.amount}")